Importando Pacotes

In [3]:
# !pip install -r requirements.txt

In [1]:
from joblib import load
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import  MinMaxScaler

Configurações do codigo

In [2]:
season = 2023

### Aplicação do Modelo

In [3]:
model = load(f'modelos/modelo_arvore_decisao_brasileirao_{season}.joblib')

In [4]:
def generate_df_feature_label(df:pd.DataFrame):

    _list = ['teams.home.id', 'score.halftime.home', 'home.pf.drw', 'home.pf.win', 'home.pf.los',
             'home.BLPS', 'home.BSH', 'home.CK', 'home.EXPGL', 'home.GKSV',
             'home.OFSD', 'home.RCD', 'home.SHFG', 'home.SHIN',
             'home.SHOG', 'home.SHOU'
          ]
    list_features = _list + [x.replace('home', 'away') for x in _list] + ['HTR']

    features = df[list_features]
    features = MinMaxScaler().fit(features).transform(features)
    labels = df['result']

    return features, labels

In [7]:
df_previsao = pd.read_excel('data/trusted/base_brasileirao_2023.xlsx')
df_previsao = df_previsao[df_previsao['league.round'] >= 28]

In [9]:
X, y = generate_df_feature_label(df_previsao)
X = MinMaxScaler().fit(X).transform(X)

In [10]:
df_previsao['previsao'] = model.predict(X)

df_previsao['previsao_time'] = \
    np.where( df_previsao['previsao'] == 1, df_previsao['teams.home.name'],
    np.where( df_previsao['previsao'] == 2, df_previsao['teams.away.name'],
    np.where( df_previsao['previsao'] == 0, 'Empate',
    None)))

df_previsao['halftime_score'] = df_previsao[["score.halftime.home", "score.halftime.away"]]\
                                    .astype('str')\
                                    .apply("x".join, axis=1)

df_previsao['fulltime_score'] = df_previsao[["score.fulltime.home", "score.fulltime.away"]]\
                                    .astype('str')\
                                    .apply("x".join, axis=1)

c:\Users\BlueShift\miniconda3\envs\infnetSports\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


In [11]:
df_previsao = df_previsao[[
    'fixture.date', 'league.round', 'teams.home.name', 'teams.away.name',
    'previsao_time', 'result.team', 'halftime_score', 'fulltime_score',
    'home.pf.drw', 'home.pf.win', 'home.pf.los', 'away.pf.drw', 'away.pf.win',
    'away.pf.los', 'home.BLPS', 'home.BSH', 'home.CK', 'home.EXPGL', 'home.FOU',
    'home.GKSV', 'home.OFSD', 'home.PAS%', 'home.PASAC', 'home.RCD', 'home.SHFG',
    'home.SHIN', 'home.SHOG', 'home.SHOU', 'home.TPAS', 'home.TSH', 'home.YCD',
    'away.BLPS', 'away.BSH', 'away.CK', 'away.EXPGL', 'away.FOU',
    'away.GKSV', 'away.OFSD', 'away.PAS%', 'away.PASAC', 'away.RCD',
    'away.SHFG', 'away.SHIN', 'away.SHOG', 'away.SHOU', 'away.TPAS',
    'away.TSH', 'away.YCD'
]]

In [15]:
df_previsao.head()

,fixture.date,league.round,teams.home.name,teams.away.name,previsao_time,result.team,halftime_score,fulltime_score,home.pf.drw,home.pf.win,...,away.PAS%,away.PASAC,away.RCD,away.SHFG,away.SHIN,away.SHOG,away.SHOU,away.TPAS,away.TSH,away.YCD
270,2023-10-22,28,Flamengo,Vasco DA Gama,Flamengo,Flamengo,0x0,1x0,0.33,0.33,...,83,353,0,4,12,7,5,427,17,2
271,2023-10-21,28,Botafogo,Atletico Paranaense,Botafogo,Empate,1x1,1x1,0.33,0.33,...,77,156,0,4,2,1,4,202,6,1
272,2023-10-21,28,Sao Paulo,Gremio,Sao Paulo,Sao Paulo,1x0,3x0,0.00,0.67,...,71,237,0,0,0,2,3,333,3,3
273,2023-10-22,28,Corinthians,America Mineiro,Corinthians,Empate,0x1,1x1,0.67,0.33,...,74,200,0,8,5,4,8,270,13,1
274,2023-10-22,28,RB Bragantino,Fluminense,RB Bragantino,RB Bragantino,1x0,1x0,0.00,1.00,...,84,426,0,4,8,5,4,509,12,2


### Output Resultados

In [14]:
df_previsao.to_excel(
    f'data/refined/previsao_brasileirao_{season}/predict_{datetime.now().strftime('%y%m%d %H%M%S')}.xlsx',
    index=False
)